# Counterfactual Logic Examples

This notebook demonstrates counterfactual reasoning using the Logos theory framework. We explore both **would-counterfactuals** (□→) and **might-counterfactuals** (◇→), showing both valid theorems and invalid principles through countermodels.

## Overview

Counterfactual conditionals express claims about what would or might be the case if things were different. Unlike material conditionals, counterfactuals:
- Use alternative worlds semantics
- Are non-monotonic (fail antecedent strengthening)
- Are non-contrapositive
- Support fine-grained hyperintensional distinctions

## Contents
1. **Setup** - Load the counterfactual theory and operators
2. **Would-Counterfactual Countermodel** - Antecedent strengthening failure
3. **Would-Counterfactual Theorem** - Simplification of disjunctive antecedents
4. **Might-Counterfactual Countermodel** - Contraposition failure
5. **Might-Counterfactual Theorem** - Possibility distribution

In [1]:
import sys
from model_checker.jupyter import create_build_example
from model_checker.theory_lib.logos.semantic import LogosSemantics, LogosProposition, LogosModelStructure
from model_checker.theory_lib.logos.operators import LogosOperatorRegistry

# Create operator registry for counterfactual theory
# We need modal operators as well since counterfactuals build on modal logic
counterfactual_registry = LogosOperatorRegistry()
counterfactual_registry.load_subtheories(['extensional', 'modal', 'counterfactual'])

# Build the semantic theory dictionary
cf_theory = {
    "semantics": LogosSemantics,
    "proposition": LogosProposition,
    "model": LogosModelStructure,
    "operators": counterfactual_registry.get_operators(),
}

print("=" * 70)
print("COUNTERFACTUAL THEORY LOADED")
print("=" * 70)
print("\nAvailable counterfactual operators:")
print("  - Would-counterfactual: \\\\boxright (□→)")
print("  - Might-counterfactual: \\\\diamondright (◇→)")
print("\nThe theory also includes classical and modal operators for complex formulas.")
print("=" * 70)

COUNTERFACTUAL THEORY LOADED

Available counterfactual operators:
  - Would-counterfactual: \\boxright (□→)
  - Might-counterfactual: \\diamondright (◇→)

The theory also includes classical and modal operators for complex formulas.


## Example 1: Would-Counterfactual Countermodel (Antecedent Strengthening Fails)

### Background
One of the most important properties of counterfactual conditionals is that they are **non-monotonic** - adding information to the antecedent can change the truth value. This is called the failure of **antecedent strengthening**.

### The Argument
We test whether the following inference is valid:
- **Premises**: 
  1. ¬A (A is actually false)
  2. A □→ C (If A were true, C would be true)
- **Conclusion**: (A ∧ B) □→ C (If both A and B were true, C would be true)

In classical logic, strengthening the antecedent preserves truth. But counterfactuals behave differently because the alternative worlds for "A" might differ from those for "A and B".

In [2]:
# CF_CM_1: Test antecedent strengthening for would-counterfactuals
CF_CM_1_example = [
    ['\\neg A', '(A \\boxright C)'],      # Premises
    ['((A \\wedge B) \\boxright C)'],     # Conclusion
    {
        'N': 4,
        'contingent': True,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 10,
        'iterate': 2,
        'expectation': True,  # We expect to find a countermodel
    }
]

print("Running model checker...")
model = create_build_example('CF_CM_1', cf_theory, CF_CM_1_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'CF_CM_1',
        'Would-Counterfactual Antecedent Strengthening',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected - the argument should be invalid)")

Running model checker...

EXAMPLE CF_CM_1: there is a countermodel.

Atomic States: 4

Semantic Theory: Would-Counterfactual Antecedent Strengthening

Premises:
1. \neg A
2. (A \boxright C)

Conclusion:
3. ((A \wedge B) \boxright C)

Z3 Run Time: 0.0322 seconds


State Space:
  #b0000 = □
  #b0001 = a
  #b0010 = b
  #b0011 = a.b (impossible)
  #b0100 = c
  #b0101 = a.c (world)
  #b0110 = b.c (world)
  #b0111 = a.b.c (impossible)
  #b1000 = d
  #b1001 = a.d (world)
  #b1010 = b.d (impossible)
  #b1011 = a.b.d (impossible)
  #b1100 = c.d (impossible)
  #b1101 = a.c.d (impossible)
  #b1110 = b.c.d (impossible)
  #b1111 = a.b.c.d (impossible)

The evaluation world is: b.c

INTERPRETED PREMISES:

1.  |\neg A| = < {b.c}, {a, a.b.c.d} >  (True in b.c)
      |A| = < {a, a.b.c.d}, {b.c} >  (False in b.c)

2.  |(A \boxright C)| = < {a.c, b.c}, {a.d} >  (True in b.c)
      |A| = < {a, a.b.c.d}, {b.c} >  (False in b.c)
      |A|-alternatives to b.c = {a.c}
        |C| = < {a.c}, {a.b.c.d, a.b.d, a

### Result Interpretation

The countermodel above demonstrates that antecedent strengthening fails for counterfactuals. The model shows:
- The premises are true: A is false in the actual world, and if A were true, C would be true
- The conclusion is false: If both A and B were true, C would NOT necessarily be true

This happens because the alternative worlds for "A" are different from those for "A and B". Adding B changes which worlds are considered as alternatives.

---

## Example 2: Would-Counterfactual Theorem (Simplification of Disjunctive Antecedents)

### Background
While counterfactuals fail antecedent strengthening, they do validate **simplification of disjunctive antecedents**. This principle states that if a disjunction would lead to a consequence, then each disjunct individually would also lead to that consequence.

### The Argument
We test whether the following inference is valid:
- **Premise**: (A ∨ B) □→ C (If either A or B were true, C would be true)
- **Conclusion**: A □→ C (If A were true, C would be true)

This is valid because any world that makes A true also makes (A ∨ B) true, so the alternative worlds for A are a subset of those for (A ∨ B).

In [3]:
# CF_TH_5: Test simplification of disjunctive antecedents
CF_TH_5_example = [
    ['((A \\vee B) \\boxright C)'],    # Premise
    ['(A \\boxright C)'],               # Conclusion
    {
        'N': 4,
        'contingent': False,
        'disjoint': False,
        'non_empty': False,
        'non_null': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': False,  # We expect NO countermodel (theorem is valid)
    }
]

print("Running model checker...")
model = create_build_example('CF_TH_5', cf_theory, CF_TH_5_example)

# Display result
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to a theorem!")
    model.model_structure.print_to(
        model.settings,
        'CF_TH_5',
        'Simplification of Disjunctive Antecedents',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Simplification of Disjunctive Antecedents")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nThis confirms that: ((A ∨ B) □→ C) ⊨ (A □→ C)")
    print("If either A or B would lead to C, then A alone would lead to C.")
    print("=" * 70)

Running model checker...
THEOREM VALIDATED: Simplification of Disjunctive Antecedents
No countermodel found - the inference is VALID

This confirms that: ((A ∨ B) □→ C) ⊨ (A □→ C)
If either A or B would lead to C, then A alone would lead to C.


### Result Interpretation

This theorem shows an important asymmetry in counterfactual logic:
- **Strengthening fails**: (A □→ C) ⊭ ((A ∧ B) □→ C)
- **Simplification succeeds**: ((A ∨ B) □→ C) ⊨ (A □→ C)

This asymmetry reflects how alternative worlds are selected based on the antecedent's content.

---

## Example 3: Might-Counterfactual Countermodel (Contraposition Fails)

### Background
Might-counterfactuals (◇→) express what might be the case if the antecedent were true. They are defined as: A ◇→ B ≡ ¬(A □→ ¬B). Like would-counterfactuals, they fail **contraposition** - a principle valid in classical logic but not for counterfactuals.

### The Argument
We test whether the following inference is valid:
- **Premise**: A □→ B (If A were true, B would be true)  
- **Conclusion**: ¬B □→ ¬A (If B were false, A would be false)

Contraposition states that if A implies B, then not-B implies not-A. This works for material conditionals but fails for counterfactuals because the alternative worlds depend on the specific antecedent.

In [4]:
# CF_CM_7: Test contraposition for counterfactuals
CF_CM_7_example = [
    ['(A \\boxright B)'],                # Premise
    ['(\\neg B \\boxright \\neg A)'],    # Conclusion
    {
        'N': 4,
        'contingent': True,
        'non_null': True,
        'non_empty': True,
        'disjoint': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': True,  # We expect to find a countermodel
    }
]

print("Running model checker...")
model = create_build_example('CF_CM_7', cf_theory, CF_CM_7_example)

# Display the countermodel if found
if model.model_structure.z3_model:
    model.model_structure.print_to(
        model.settings,
        'CF_CM_7',
        'Counterfactual Contraposition',
        output=sys.stdout
    )
else:
    print("No countermodel found (unexpected - contraposition should fail)")

Running model checker...

EXAMPLE CF_CM_7: there is a countermodel.

Atomic States: 4

Semantic Theory: Counterfactual Contraposition

Premise:
1. (A \boxright B)

Conclusion:
2. (\neg B \boxright \neg A)

Z3 Run Time: 0.0122 seconds


State Space:
  #b0000 = □
  #b0001 = a
  #b0010 = b
  #b0011 = a.b
  #b0100 = c
  #b0101 = a.c
  #b0110 = b.c
  #b0111 = a.b.c (world)
  #b1000 = d
  #b1001 = a.d
  #b1010 = b.d
  #b1011 = a.b.d (world)
  #b1100 = c.d
  #b1101 = a.c.d (impossible)
  #b1110 = b.c.d (world)
  #b1111 = a.b.c.d (impossible)

The evaluation world is: a.b.d

INTERPRETED PREMISE:

1.  |(A \boxright B)| = < {a.b.d}, {a.b.c, b.c.d} >  (True in a.b.d)
      |A| = < {b.d}, {a.b.c} >  (True in a.b.d)
      |A|-alternatives to a.b.d = {a.b.d}
        |B| = < {a.b.d}, {a.b.c, a.b.c.d, a.c.d, b.c.d, c} >  (True in a.b.d)

INTERPRETED CONCLUSION:

2.  |(\neg B \boxright \neg A)| = < {}, {a.b.c, a.b.d, b.c.d} >  (False in a.b.d)
      |\neg B| = < {a.b.c, a.b.c.d, a.c.d, b.c.d, c}, {a.b.

### Result Interpretation

The countermodel demonstrates that contraposition fails for counterfactuals. The model shows:
- The premise is true: If A were true, B would be true  
- The conclusion is false: It's NOT the case that if B were false, A would be false

This failure occurs because the worlds selected as alternatives when we suppose ¬B might be completely different from those selected when we suppose A. The counterfactual depends on the specific content of the antecedent.

---

## Example 4: Might-Counterfactual Theorem (Factivity)

### Background
While might-counterfactuals fail many classical principles, they do satisfy **factivity** - if both A and B are actually true, then it follows that if A were true, B might be true. This is a weaker principle than the corresponding one for would-counterfactuals.

### The Argument
We test whether the following inference is valid:
- **Premises**: 
  1. A (A is actually true)
  2. B (B is actually true)
- **Conclusion**: A ◇→ B (If A were true, B might be true)

This is valid because the actual world itself serves as an alternative world where both A and B are true.

In [5]:
# CF_TH_10: Test might-counterfactual factivity
CF_TH_10_example = [
    ['A', 'B'],                         # Premises
    ['(A \\diamondright B)'],           # Conclusion
    {
        'N': 4,
        'contingent': False,
        'disjoint': False,
        'non_empty': False,
        'non_null': False,
        'max_time': 10,
        'iterate': 1,
        'expectation': False,  # We expect NO countermodel (theorem is valid)
    }
]

print("Running model checker...")
model = create_build_example('CF_TH_10', cf_theory, CF_TH_10_example)

# Display result
if model.model_structure.z3_model:
    print("UNEXPECTED: Found a countermodel to a theorem!")
    model.model_structure.print_to(
        model.settings,
        'CF_TH_10',
        'Might-Counterfactual Factivity',
        output=sys.stdout
    )
else:
    print("=" * 70)
    print("THEOREM VALIDATED: Might-Counterfactual Factivity")
    print("=" * 70)
    print("No countermodel found - the inference is VALID")
    print("\nThis confirms that: A, B ⊨ (A ◇→ B)")
    print("If A and B are both true, then if A were true, B might be true.")
    print("\nThis follows because the actual world serves as a witness.")
    print("=" * 70)

Running model checker...
THEOREM VALIDATED: Might-Counterfactual Factivity
No countermodel found - the inference is VALID

This confirms that: A, B ⊨ (A ◇→ B)
If A and B are both true, then if A were true, B might be true.

This follows because the actual world serves as a witness.


### Result Interpretation

This theorem illustrates the relationship between actuality and possibility in counterfactual reasoning:
- What is actually true is also counterfactually possible
- The actual world always counts as one of the alternative worlds

This provides a connection between the actual world and counterfactual alternatives.

---

## Summary

These examples demonstrate key properties of counterfactual logic:

### Would-Counterfactuals (□→)
- **Fail antecedent strengthening**: Adding to the antecedent can change truth value
- **Validate disjunction simplification**: Disjunctive antecedents can be simplified
- **Non-contrapositive**: Contraposition is invalid

### Might-Counterfactuals (◇→)  
- **Also fail contraposition**: Like would-counterfactuals
- **Satisfy factivity**: Actual truth implies counterfactual possibility
- **Defined via would-counterfactuals**: A ◇→ B ≡ ¬(A □→ ¬B)

These properties make counterfactual logic suitable for modeling hypothetical reasoning, causal claims, and fine-grained semantic distinctions that classical logic cannot capture.